In [1]:
# Get the data for expeirment
import sklearn
import pandas as pd 
import numpy as np

from sklearn import tree
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import explained_variance_score, mean_squared_error
from scipy.spatial.distance import cdist
from numpy.linalg import pinv
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import geopy.distance as distance
from sklearn.preprocessing import MinMaxScaler

import Data_utils

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [35]:
# training = pd.read_csv('Data/ab_wind_train_data.txt')
# testing = pd.read_csv('Data/ab_wind_test_data.txt')

training = pd.read_csv('Data\processed_ab_wind.csv')

C:\Users\PPL\AppData\Local\Temp\ipykernel_59264\2678437863.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  training = pd.read_csv('Data\processed_ab_wind.csv')


In [36]:
training

,longitude,latitude,name,id,date,temp,precip,wind_direction,wind_speed
0,-112.82,53.05,CAMROSE,3011240,2013-12-10 16:00,-21.6,0.0,24.0,6.0
1,-112.82,53.05,CAMROSE,3011240,2013-12-10 17:00,-20.9,0.0,19.0,6.0
2,-112.82,53.05,CAMROSE,3011240,2013-12-10 18:00,-21.4,0.0,19.0,9.0
3,-112.82,53.05,CAMROSE,3011240,2013-12-10 19:00,-22.5,0.0,15.0,10.0
4,-112.82,53.05,CAMROSE,3011240,2013-12-10 20:00,-21.3,0.0,16.0,13.0
...,...,...,...,...,...,...,...,...,...
2560209,-115.28,56.55,RED EARTH,3075488,2023-07-21 07:00,13.9,0.3,13.0,4.0
2560210,-115.28,56.55,RED EARTH,3075488,2023-07-21 08:00,17.0,0.3,13.0,1.0
2560211,-115.28,56.55,RED EARTH,3075488,2023-07-21 10:00,22.6,0.0,21.0,2.0
2560212,-115.28,56.55,RED EARTH,3075488,2023-07-21 11:00,24.1,0.0,28.0,3.0


In [37]:
# Rest reserved as D0
training = training.loc[training['date'] <= '2022-01-01']

In [40]:
training

,longitude,latitude,name,id,date,temp,precip,wind_direction,wind_speed
0,-112.82,53.05,CAMROSE,3011240,2013-12-10 16:00,-21.6,0.0,24.0,6.0
1,-112.82,53.05,CAMROSE,3011240,2013-12-10 17:00,-20.9,0.0,19.0,6.0
2,-112.82,53.05,CAMROSE,3011240,2013-12-10 18:00,-21.4,0.0,19.0,9.0
3,-112.82,53.05,CAMROSE,3011240,2013-12-10 19:00,-22.5,0.0,15.0,10.0
4,-112.82,53.05,CAMROSE,3011240,2013-12-10 20:00,-21.3,0.0,16.0,13.0
...,...,...,...,...,...,...,...,...,...
2547010,-115.28,56.55,RED EARTH,3075488,2021-12-31 19:00,-28.4,0.0,13.0,7.0
2547011,-115.28,56.55,RED EARTH,3075488,2021-12-31 20:00,-27.3,0.0,13.0,9.0
2547012,-115.28,56.55,RED EARTH,3075488,2021-12-31 21:00,-26.9,0.0,15.0,8.0
2547013,-115.28,56.55,RED EARTH,3075488,2021-12-31 22:00,-27.0,0.0,14.0,9.0


In [78]:
training_results = []
validation_results = []
test_results = []

In [79]:
# Hyperparameters

# General hyperparameter
number_of_neighbour_station = 3
max_distance_threshold = 100 #km

train_test_cutoff = 0.7 # 7:3 train test
train_validation_cutoff = 0.7 # 7 : 3 train validation

# NN hyperparameters
number_of_epochs = 30

# Fcm hyperparameters
FCM_Nc = 30
FCM_m = 2.0
tol = 1e-5
max_iter = 100

In [80]:
target_stations = [
    "CALGARY INT'L CS",
    "EDMONTON BLATCHFORD",
    "EDSON CLIMATE",
    "PINCHER CREEK CLIMATE",
    "LETHBRIDGE CDA",
]

# target_stations = [
#     "EDSON CLIMATE"
# ]

In [81]:
# Source: From the TSmodel
def preprocess_data(train, validation, test):
    scaler = MinMaxScaler()
    train_data = scaler.fit_transform(train)
    validation_data = scaler.transform(validation)
    test_data = scaler.transform(test)
    
    return train_data, validation_data, test_data

In [82]:
def fcm_rule_model(train_x, train_y, validation_x, validation_y, test_x, test_y):
    N = train_x.shape[0]
    U = np.random.rand(N, FCM_Nc)
    U = U / np.sum(U, axis=1, keepdims=True)

    # Fuzzy C-Means 聚类
    for _ in range(max_iter):
        U_old = U.copy()
        mf = U ** FCM_m
        cen = np.dot(mf.T, train_x) / np.sum(mf, axis=0, keepdims=True).T
        dists = cdist(train_x, cen)
        tmp = np.power(dists, -2 / (FCM_m - 1))
        U = tmp / np.sum(tmp, axis=1, keepdims=True)

        # 检查是否收敛
        if np.linalg.norm(U - U_old) < tol:
            break

    # 模型训练
    z = np.hstack((np.ones((N, 1)), train_x))
    H = np.zeros((N, FCM_Nc * (train_x.shape[1] + 1)))

    # 生成模糊规则矩阵 H
    for j in range(FCM_Nc):
        Uj = np.tile(U[:, j], (train_x.shape[1] + 1, 1)).T
        H[:, (j * (train_x.shape[1] + 1)):((j + 1) * (train_x.shape[1] + 1))] = Uj * z

    # 计算权重 W
    W = pinv(np.dot(H.T, H)).dot(H.T).dot(train_y)

    # 在训练集上进行预测
    y_hat_train = np.dot(H, W)
    mse_train = mean_squared_error(train_y, y_hat_train)
    rmse_train = sqrt(mean_squared_error(train_y, y_hat_train))
    print("训练集的MSE:", mse_train)
    print("训练集的RMSE:", rmse_train)
    training_results.append({'fcm_train':rmse_train})
    
    # validation
    N_validation = validation_x.shape[0]
    z_validation = np.hstack((np.ones((N_validation, 1)), validation_x))
    H_validation = np.zeros((N_validation, FCM_Nc * (validation_x.shape[1] + 1)))
    dists_validation = cdist(validation_x, cen)
    tmp_validation = np.power(dists_validation, -2 / (FCM_m - 1))
    U_validation = tmp_validation / np.sum(tmp_validation, axis=1, keepdims=True)

    # 生成模糊规则矩阵 H_test
    for j in range(FCM_Nc):
        Uj_validation = np.tile(U_validation[:, j], (validation_x.shape[1] + 1, 1)).T
        H_validation[:, (j * (validation_x.shape[1] + 1)):((j + 1) * (validation_x.shape[1] + 1))] = Uj_validation * z_validation

    # validation set
    y_hat_validation = np.dot(H_validation, W)
    mse_validation = mean_squared_error(validation_y, y_hat_validation)
    rmse_validation = sqrt(mse_validation)
    print("测试集的MSE:", mse_validation)
    print("测试集的RMSE:", rmse_validation)
    validation_results.append({'fcm_validation':rmse_validation})

    # 模型测试
    N_test = test_x.shape[0]
    z_test = np.hstack((np.ones((N_test, 1)), test_x))
    H_test = np.zeros((N_test, FCM_Nc * (test_x.shape[1] + 1)))
    dists_test = cdist(test_x, cen)
    tmp_test = np.power(dists_test, -2 / (FCM_m - 1))
    U_test = tmp_test / np.sum(tmp_test, axis=1, keepdims=True)

    # 生成模糊规则矩阵 H_test
    for j in range(FCM_Nc):
        Uj_test = np.tile(U_test[:, j], (test_x.shape[1] + 1, 1)).T
        H_test[:, (j * (test_x.shape[1] + 1)):((j + 1) * (test_x.shape[1] + 1))] = Uj_test * z_test

    # 在测试集上进行预测
    y_hat_test = np.dot(H_test, W)
    mse_test = mean_squared_error(test_y, y_hat_test)
    rmse_test = sqrt(mean_squared_error(test_y, y_hat_test))
    print("测试集的MSE:", mse_test)
    print("测试集的RMSE:", rmse_test)
    test_results.append({'fcm_test':rmse_test})
    
        
    print('r2 score')
    print(r2_score(test_y, y_hat_test))


    # 绘制拟合曲线
#     plt.figure()
#     plt.subplot(1, 2, 1)
#     plt.scatter(range(len(train_y)), train_y, c='blue', label='Target')
#     plt.scatter(range(len(y_hat_train)), y_hat_train, c='red', label='Predict')
#     plt.title('Training')
#     plt.legend()

#     plt.subplot(1, 2, 2)
#     plt.scatter(range(len(test_y)), test_y, c='blue', label='Target')
#     plt.scatter(range(len(y_hat_test)), y_hat_test, c='red', label='Predict')
#     plt.title('Testing')
#     plt.legend()
#     plt.show()

In [83]:
def linear_model(train_x, train_y, validation_x, validation_y, test_x, test_y):
    reg = LinearRegression().fit(train_x, train_y)
    
    validation_pred = reg.predict(validation_x)
    validation_results.append({'linear_validation':mean_squared_error(validation_y, validation_pred, squared = False)})
    
    test_pred = reg.predict(test_x)
    test_results.append({'linear_validation':mean_squared_error(test_y, test_pred, squared = False)})
    
    print("linear_validation")
    print(mean_squared_error(validation_y, validation_pred, squared = False))
    print("linear_test")
    print(mean_squared_error(test_y, test_pred, squared = False))

In [84]:
def build_neural_network_model():
    model = models.Sequential()
    
    model.add(layers.Dense(100, activation='relu'))
    
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))    
    
    return model

In [85]:
def neural_network_model(train_x, train_y, validation_x, validation_y, test_x, test_y):
    neural_network_model = build_neural_network_model()
    
    neural_network_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    
    history = neural_network_model.fit(
        train_x, 
        train_y, 
        epochs=number_of_epochs, 
        validation_data=(validation_x, validation_y)
    )
    
    # Record the result for each of 10 folds
    training_results.append({
        'nn_train' : np.mean(np.array(history.history['root_mean_squared_error']))
    })
    validation_results.append({
        'nn_validation' : np.mean(np.array(history.history['val_root_mean_squared_error']))
    })
    
    # Test the model
    test_result = neural_network_model.evaluate(test_x, test_y, return_dict = True)
    test_results.append({
        'nn_test' : np.mean(np.array(test_result['root_mean_squared_error']))
    })
    
    print('nn_test')
    print(test_result['root_mean_squared_error'])

In [86]:
train_x = []
train_y = []
validation_x = []
validation_y = []
test_x = []
test_y = []

for station in target_stations:
    features, target = Data_utils.generate_data(training, station, number_of_neighbour_station, max_distance_threshold)
    
    np_features = np.array(features)
    np_target = np.array(target)
    
    number_of_samples = np_features.shape[0] # first dimension
    
    test_cutoff_index = np.floor(number_of_samples * train_test_cutoff).astype(int)
    validation_cutoff_index = np.floor(test_cutoff_index * train_validation_cutoff).astype(int)
    
    temp_train_x = np_features[:validation_cutoff_index]
    temp_train_y = np_target[:validation_cutoff_index]
    
    temp_validation_x = np_features[validation_cutoff_index:test_cutoff_index]
    temp_validation_y = np_target[validation_cutoff_index:test_cutoff_index]

    temp_test_x = np_features[test_cutoff_index:]
    temp_test_y = np_target[test_cutoff_index:]
    
    train_x.append(temp_train_x)
    train_y.append(temp_train_y)
    validation_x.append(temp_validation_x)
    validation_y.append(temp_validation_y)
    test_x.append(temp_test_x)
    test_y.append(temp_test_y)

CALGARY INT'L CS
        longitude  latitude              name       id              date  \
863794     -114.0     51.11  CALGARY INT'L CS  3031094  2015-12-21 17:00   
863795     -114.0     51.11  CALGARY INT'L CS  3031094  2015-12-21 18:00   
863796     -114.0     51.11  CALGARY INT'L CS  3031094  2015-12-21 20:00   
863797     -114.0     51.11  CALGARY INT'L CS  3031094  2015-12-21 22:00   
863798     -114.0     51.11  CALGARY INT'L CS  3031094  2015-12-21 23:00   
...           ...       ...               ...      ...               ...   
914052     -114.0     51.11  CALGARY INT'L CS  3031094  2021-12-31 19:00   
914053     -114.0     51.11  CALGARY INT'L CS  3031094  2021-12-31 20:00   
914054     -114.0     51.11  CALGARY INT'L CS  3031094  2021-12-31 21:00   
914055     -114.0     51.11  CALGARY INT'L CS  3031094  2021-12-31 22:00   
914056     -114.0     51.11  CALGARY INT'L CS  3031094  2021-12-31 23:00   

        temp  precip  wind_direction  wind_speed  
863794  -4.9     0.

LETHBRIDGE CDA
         longitude  latitude            name       id              date  temp  \
1085086    -112.77      49.7  LETHBRIDGE CDA  3033890  2013-12-10 16:00 -13.8   
1085087    -112.77      49.7  LETHBRIDGE CDA  3033890  2013-12-10 17:00 -14.4   
1085088    -112.77      49.7  LETHBRIDGE CDA  3033890  2013-12-10 18:00 -17.7   
1085089    -112.77      49.7  LETHBRIDGE CDA  3033890  2013-12-10 19:00 -19.3   
1085090    -112.77      49.7  LETHBRIDGE CDA  3033890  2013-12-10 20:00 -21.9   
...            ...       ...             ...      ...               ...   ...   
1143883    -112.77      49.7  LETHBRIDGE CDA  3033890  2020-11-14 00:00   2.9   
1143884    -112.77      49.7  LETHBRIDGE CDA  3033890  2020-11-15 00:00  -3.7   
1143885    -112.77      49.7  LETHBRIDGE CDA  3033890  2020-11-16 00:00   2.6   
1143886    -112.77      49.7  LETHBRIDGE CDA  3033890  2020-11-17 00:00   2.1   
1143887    -112.77      49.7  LETHBRIDGE CDA  3033890  2020-11-17 23:00   6.0   

         pre

In [87]:
train_x = np.array([data for sublist in train_x for data in sublist])
train_y = np.array([data for sublist in train_y for data in sublist])

validation_x = np.array([data for sublist in validation_x for data in sublist])
validation_y = np.array([data for sublist in validation_y for data in sublist])

test_x = np.array([data for sublist in test_x for data in sublist])
test_y = np.array([data for sublist in test_y for data in sublist])

In [88]:
train_x, validation_x, test_x = preprocess_data(train_x, validation_x, test_x)    

In [89]:
# Linear model, as a reference line
linear_model(train_x, train_y, validation_x, validation_y, test_x, test_y)

linear_validation
7.8591381609995095
linear_test
7.632462685921033


In [90]:
# FCM rule based model 
fcm_rule_model(train_x, train_y, validation_x, validation_y, test_x, test_y)

训练集的MSE: 46.64798124743017
训练集的RMSE: 6.829932741061962
测试集的MSE: 52.18323330724718
测试集的RMSE: 7.223796322381133
测试集的MSE: 49.9625751002685
测试集的RMSE: 7.068420976446473
r2 score
0.6414428848912515


In [91]:
# NN network
neural_network_model(train_x, train_y, validation_x, validation_y, test_x, test_y)

Epoch 1/30
1821/1821 [==============================] - 7s 3ms/step - loss: 74.8451 - root_mean_squared_error: 8.6513 - val_loss: 61.8377 - val_root_mean_squared_error: 7.8637
Epoch 2/30
1821/1821 [==============================] - 6s 4ms/step - loss: 56.2046 - root_mean_squared_error: 7.4970 - val_loss: 60.7554 - val_root_mean_squared_error: 7.7946
Epoch 3/30
1821/1821 [==============================] - 6s 3ms/step - loss: 55.0183 - root_mean_squared_error: 7.4174 - val_loss: 59.1503 - val_root_mean_squared_error: 7.6909
Epoch 4/30
1821/1821 [==============================] - 6s 3ms/step - loss: 53.6203 - root_mean_squared_error: 7.3226 - val_loss: 57.6072 - val_root_mean_squared_error: 7.5899
Epoch 5/30
1821/1821 [==============================] - 7s 4ms/step - loss: 52.4066 - root_mean_squared_error: 7.2392 - val_loss: 57.0724 - val_root_mean_squared_error: 7.5546
Epoch 6/30
1821/1821 [==============================] - 6s 3ms/step - loss: 51.2778 - root_mean_squared_error: 7.1608 - 

In [94]:
test_results

[{'linear_validation': 7.632462685921033},
 {'fcm_test': 7.068420976446473},
 {'nn_test': 6.983702659606934}]

In [95]:
validation_results

[{'linear_validation': 7.8591381609995095},
 {'fcm_validation': 7.223796322381133},
 {'nn_validation': 7.307941468556722}]